In [1]:
import pandas as pd
import numpy as np
import datetime
from openpyxl import load_workbook

In [38]:
SEASON = 13
MATCHDAY = 9

In [39]:
mc = pd.read_excel('data/match_'+str(MATCHDAY)+'.xlsx')
mc.drop('№', axis=1, inplace=True)
mc.rename(columns={'index': 'Name'}, inplace=True)

In [24]:
mc.columns

Index(['Name', 'Итог', 'вольфсбург – боруссия м', '1\n3',
       'штутгарт – хоффенхайм', '3\n1', 'цска – краснодар', '0\n0',
       'брайтон – атрсенал', '0\n0.1', 'атлетико – барселона', '2\n0',
       'андерлехт – брюгге', '1\n1', 'лилль – марсель', '2\n0.1',
       'ливерпуль – манчестер сити', '2\n2', 'динамо киев – шахтер', '0\n0.2',
       'аталанта – милан', '2\n3'],
      dtype='object')

In [40]:
#Заголовки нулевой таблицы для создания ДБ ставок
zeros_col = ['Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']

matches = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='matches', index_col=0)
players = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='players', index_col=0)
season = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='season', index_col=0)
teams = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='teams', index_col=0)

In [41]:
def proceed_bet(player_id, match_id):
    """
    proceed_bet(player_id, match_id)
    Сверяет ставки с результатом матча.
    player_id = Id участника
    match_id = Id матча
    """
    player_name = players['Name'].iloc[player_id]
    
    h_team = matches['Hometeam'].loc[match_id]
    a_team = matches['Awayteam'].loc[match_id]
    match = str(h_team)+' - '+str(a_team)
    
    
    try:
        hscore_true = matches['H_score'].loc[match_id]
        ascore_true = matches['A_score'].loc[match_id]

        hscore_bet = int(bets['bet_hscore_'+str(match_id)].loc[player_id])
        ascore_bet = int(bets['bet_ascore_'+str(match_id)].loc[player_id])
        
    except ValueError:
        print(player_name, 'made no bet on', match)
        bets['points_'+str(match_id)].loc[player_id] = 0
        return
    
    true_score= str(hscore_true)+':'+str(ascore_true)
    bet_score= str(hscore_bet)+':'+str(ascore_bet)
    
    points_granted = 0
    
    if (hscore_true == hscore_bet and ascore_true == ascore_bet):
        points_granted = 5
        matches['Exact Score'].loc[match_id] += 1
        season['Exact Score'].loc[player_id] += 1
        bets['Exact Score'].loc[player_id] += 1
    elif (hscore_true - ascore_true) == (hscore_bet - ascore_bet):
        points_granted = 3
        matches['Right Margin'].loc[match_id] += 1
        season['Right Margin'].loc[player_id] += 1
        bets['Right Margin'].loc[player_id] += 1
    elif ((hscore_true > ascore_true) and (hscore_bet > ascore_bet)) or ((hscore_true < ascore_true) and (hscore_bet < ascore_bet)):
        points_granted = 2
        matches['Right Result'].loc[match_id] += 1
        season['Right Result'].loc[player_id] += 1
        bets['Right Result'].loc[player_id] += 1
    else:
        matches['Wrong Result'].loc[match_id] += 1
        season['Wrong Result'].loc[player_id] += 1
        bets['Wrong Result'].loc[player_id] += 1
    matches['Bets'].loc[match_id] += 1
    season['Bets'].loc[player_id] += 1
    bets['points_'+str(match_id)].loc[player_id] = points_granted
    return

def count_total(player_id):
    """
    Считает суммарное число баллов, суммируя значения с bets 
    """
    total = 0
    for i in range(11, bets.shape[1], 3):
        total += bets.loc[player_id].iloc[i]
    bets['Game Points'].loc[player_id] = total
    season['Game Points'].loc[player_id] += total
    print(bets.iloc[player_id, 0], total)
    
def find_uniques(match_ids):
    h_score = str(matches['H_score'].loc[match_ids])
    a_score = str(matches['A_score'].loc[match_ids])
    player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
    bets['Uniques'].loc[player_id] += 1
    season['Uniques'].loc[player_id] += 1
    matches['Uniques'].iloc[match_ids] += 1
    return player_id

def get_match_ids(season=SEASON, matchday=MATCHDAY):
    ids = matches[matches['Season'] == season][matches['Matchday'] == matchday].index
    return_list = ['Name', 'Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
    for i in ids:
        return_list.append('bet_hscore_'+str(i))
        return_list.append('bet_ascore_'+str(i))
        return_list.append('points_'+str(i))
    return return_list

In [42]:
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]
      
    if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
        print(f'{hometeam} - {awayteam} уже есть')
    else:
        matches = matches.append({'Hometeam': hometeam, 'Awayteam': awayteam,
                                  'H_score': int(h_score), 'A_score': int(a_score),
                                  'Season': SEASON, 'Matchday': MATCHDAY, 'Date': datetime.datetime.now(), 'Bets': 0,
                                  'Exact Score': 0, 'Right Margin': 0, 'Right Result': 0, 
                                  'Wrong Result': 0, 'Uniques': 0}, ignore_index=True)
    
bets_columns = get_match_ids(SEASON,MATCHDAY)
bets = pd.DataFrame(pd.concat([mc['Name'], pd.DataFrame(np.zeros((mc['Name'].shape[0],8)), columns=zeros_col)], axis=1), columns=bets_columns)  

#Очень тупо, я повторяю цикл, потому что между этими операциями ме надо добавить БД для ставок
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]    
    match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]   
    
    bets['bet_hscore_'+str(match_id)] = mc[match].str.split(':').str[0]
    bets['bet_ascore_'+str(match_id)] = mc[match].str.split(':').str[1]

C:\Users\ANTON~1.SHE\AppData\Local\Temp/ipykernel_17992/131283623.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
C:\Users\ANTON~1.SHE\AppData\Local\Temp/ipykernel_17992/131283623.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]


In [43]:
for player in range(0,players['Name'].shape[0]):
    for match in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
        proceed_bet(player,match)

c:\users\anton.shevchuk\appdata\local\programs\python\python39-32\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Муратов Игорь made no bet on башакшехир - бешикташ
Синичкин Александр made no bet on башакшехир - бешикташ
Шевчук Антон made no bet on башакшехир - бешикташ


In [10]:
mc

,Name,Итог,аякс – бешикташ,2\n0,шахтер – интер,0\n0,боруссия д – спортинг,1\n0,милан – атлетико,1\n2,...,бенфика – барселона,3\n0,вольфсбург – севилья,1\n1,зальцубрг – лилль,2\n1,манчестер юнайтед – вильярреал,2\n1.1,ювентус – челси,1\n0.2
0,Алферов Ян,28,3:0,2,0:3,0,3:0,2,1:1,0,...,0:1,0,1:1,5,1:0,3,2:0,2,0:0,0
1,Галкин Денис,25,3:0,2,1:3,0,2:0,2,2:0,0,...,0:2,0,1:1,5,1:1,0,3:0,2,1:2,0
2,Гришин Антон,32,2:0,5,1:2,0,2:1,3,1:1,0,...,1:1,0,1:1,5,2:1,5,2:1,5,1:2,0
3,Данилов Евгений,21,3:0,2,2:1,0,3:0,2,2:1,0,...,2:1,2,2:2,3,2:1,5,1:1,0,0:2,0
4,Дублицевич Александр,29,3:0,2,1:2,0,2:1,3,0:0,0,...,1:2,0,1:1,5,2:1,5,1:0,3,0:2,0
5,Жигалов Александр,32,1:0,2,1:2,0,2:0,2,0:1,3,...,1:0,2,1:1,5,2:1,5,0:0,0,0:1,0
6,Зубатов Михаил,25,3:0,2,1:3,0,2:0,2,2:1,0,...,1:2,0,2:1,0,2:1,5,2:1,5,1:2,0
7,Колодин Дмитрий,20,3:0,2,2:1,0,3:1,2,2:1,0,...,1:1,0,2:1,0,3:1,2,2:1,5,1:3,0
8,Конаков Никита,27,3:0,2,1:1,3,2:1,3,1:1,0,...,2:1,2,1:1,5,2:0,2,1:0,3,0:1,0
9,Криеванс Владислав,26,2:1,2,0:2,0,2:0,2,0:1,3,...,1:1,0,1:1,5,0:2,0,1:0,3,1:2,0


In [11]:
bets['Name']

0               Алферов Ян
1             Галкин Денис
2             Гришин Антон
3          Данилов Евгений
4     Дублицевич Александр
5        Жигалов Александр
6           Зубатов Михаил
7          Колодин Дмитрий
8           Конаков Никита
9       Криеванс Владислав
10        Кучеренко Никита
11        Лесик Константин
12            Малевич Егор
13         Махмудов Руслан
14           Муратов Игорь
15          Невский Леонид
16       Новиков Александр
17       Сибиряков Георгий
18      Синичкин Александр
19        Фёдоров Владимир
20          Фёдоров Михаил
21       Чапаров Александр
22            Шевчук Антон
Name: Name, dtype: object

In [10]:
def find_match_id(tour):
    return 11+(tour-1)*3

find_match_id

<function __main__.find_match_id(tour)>

In [12]:
bets.iloc[1,find_match_id(16)] = 5

In [44]:
for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
    player_id = find_uniques(i)
    bets['points_'+str(i)].loc[player_id] += 1
    
for player in range(0,players['Name'].shape[0]):
    count_total(player)

Алферов Ян 9.0
Галкин Денис 8.0
Гришин Антон 12.0
Данилов Евгений 8.0
Дублицевич Александр 11.0
Жигалов Александр 17.0
Зубатов Михаил 10.0
Колодин Дмитрий 8.0
Конаков Никита 12.0
Криеванс Владислав 12.0
Кучеренко Никита 12.0
Лесик Константин 11.0
Малевич Егор 6.0
Махмудов Руслан 17.0
Муратов Игорь 8.0
Невский Леонид 8.0
Новиков Александр 6.0
Сибиряков Георгий 11.0
Синичкин Александр 8.0
Фёдоров Владимир 10.0
Фёдоров Михаил 18.0
Чапаров Александр 15.0
Шевчук Антон 5.0


C:\Users\ANTON~1.SHE\AppData\Local\Temp/ipykernel_17992/44280827.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
C:\Users\ANTON~1.SHE\AppData\Local\Temp/ipykernel_17992/3143780336.py:70: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]


In [23]:
mc.loc[20]



Name                              Фёдоров Михаил
Итог                                          26
аякс – бешикташ                              3:0
2\n0                                           2
шахтер – интер                               1:1
0\n0                                           3
боруссия д – спортинг                        2:0
1\n0                                           2
милан – атлетико                             1:1
1\n2                                           0
порту – ливерпуль                            1:1
1\n5                                           0
псж – манчестер сити                         1:1
2\n0.1                                         0
рб лейпциг – брюгге                          1:0
1\n2.1                                         0
реал мадрид – шериф                          1:0
1\n2.2                                         0
аталанта – янг бойз                          2:1
1\n0.1                                         3
зенит – мальмё      

### Bombers

In [45]:
bombs = pd.read_excel('output/bombs_bets.xlsx', index_col=0, sheet_name=str(MATCHDAY))


In [46]:
# Сгрупировал таблицу в формате имя - очки и присвоил в новую переменную
bombs_points = bombs[bombs['Points']>0].groupby(['Name']).sum().sort_values(by=['Points'], ascending=False)

# Теперь итерируем по всей таблице bets. Если имя есть в bomb_points обновляем значение таблицы Bomb Points. 
# Важно вызывать через loc чтобы не вызывать копию, а работать напрямую с табилцей
print('Очки полученные за бомбардиров')
for row in np.arange(bets.shape[0]):
    try:
        bets.loc[row,'Bomb Points'] = bombs_points['Points'][bets.loc[row,'Name']]
        print(bets.loc[row,'Bomb Points'], bets.loc[row,'Name'])
    except KeyError:
        pass

Очки полученные за бомбардиров
2.0 Алферов Ян
4.0 Колодин Дмитрий
4.0 Конаков Никита
2.0 Лесик Константин
2.0 Невский Леонид
4.0 Сибиряков Георгий
4.0 Синичкин Александр
4.0 Фёдоров Михаил


In [47]:
bets['Total Points'] = bets['Game Points'] + bets['Bomb Points']

season['Bomb Points'] += bets['Bomb Points']
season['Total Points'] = season['Game Points'] + season['Bomb Points']

In [48]:
bets.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
20,Фёдоров Михаил,22.0
13,Махмудов Руслан,17.0
5,Жигалов Александр,17.0
8,Конаков Никита,16.0
21,Чапаров Александр,15.0
17,Сибиряков Георгий,15.0
11,Лесик Константин,13.0
2,Гришин Антон,12.0
18,Синичкин Александр,12.0
7,Колодин Дмитрий,12.0


In [49]:
season.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
0,Алферов Ян,185.0
17,Сибиряков Георгий,169.0
19,Фёдоров Владимир,165.0
11,Лесик Константин,165.0
8,Конаков Никита,163.0
15,Невский Леонид,161.0
2,Гришин Антон,159.0
10,Кучеренко Никита,159.0
4,Дублицевич Александр,152.0
7,Колодин Дмитрий,151.0


In [50]:
for match_id in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
    match = matches.iloc[match_id,:]
    hometeam, awayteam = match[0], match[1]
    m_s = matches.iloc[match_id,:]    #Match_series
    for team in [hometeam, awayteam]:
        # Какое говно)))))
        if team == m_s['Hometeam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 1,0,0
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 0,0,1
            else:
                win, draw, loss = 0,1,0
        elif team == m_s['Awayteam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 0,0,1
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 1,0,0
            else:
                win, draw, loss = 0,1,0
                      
        if team in teams['Team'].values:
            team_index = teams[teams['Team'] == team].index[0]
            teams.iloc[team_index,2] += 1
            teams.iloc[team_index,3] += win
            teams.iloc[team_index,4] += draw
            teams.iloc[team_index,5] += loss
            teams.iloc[team_index,6] += m_s['Bets']
            teams.iloc[team_index,7] += m_s['Exact Score']
            teams.iloc[team_index,8] += m_s['Right Margin']
            teams.iloc[team_index,9] += m_s['Right Result']
            teams.iloc[team_index,10] += m_s['Wrong Result']
            teams.iloc[team_index,11] += m_s['Uniques']
        else:
            teams = teams.append({'Team': team, 'Matches': 1, 'Wins': win, 'Draws': draw, 'Losses': loss,
                                 'Bets': m_s['Bets'], 'Exact Score': m_s['Exact Score'], 'Right Margin': m_s['Right Margin'], 
                                 'Right Result': m_s['Right Result'], 'Wrong Result': m_s['Wrong Result'], 
                                 'Uniques': m_s['Uniques']}, ignore_index=True)

In [32]:
teams

,Team,Country,Matches,Wins,Draws,Losses,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
0,байер,NaN,2,1,0,1,42,0,10,8,24,0
1,боруссия м,NaN,1,0,0,1,22,0,0,4,18,0
2,гранада,NaN,1,0,1,0,23,5,0,0,18,0
3,валенсия,NaN,1,0,1,0,23,5,0,0,18,0
4,локомотив,NaN,2,1,1,0,44,13,5,4,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...
78,сочи,NaN,1,0,0,1,23,2,3,0,18,0
79,динамо москва,NaN,1,1,0,0,23,2,3,0,18,0
80,стандарт,NaN,1,0,0,1,23,1,9,1,12,1
81,цска,NaN,1,1,0,0,23,4,6,2,11,0


In [51]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
book = load_workbook('output/bets.xlsx')
with pd.ExcelWriter('output/bets.xlsx', engine='openpyxl') as writer:
    writer.book = book

    # Write each dataframe to a different worksheet.
    bets.to_excel(writer, sheet_name=str(MATCHDAY))

    writer.save()

In [52]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(f'output/season_{str(SEASON)}.xlsx', engine='xlsxwriter') as writer:

    # Write each dataframe to a different worksheet.
    season.to_excel(writer, sheet_name='season')
    matches.to_excel(writer, sheet_name='matches')
    players.to_excel(writer, sheet_name='players')
    teams.to_excel(writer, sheet_name='teams')
    #scorers.to_excel(writer, sheet_name='scorers_18')

    writer.save()

c:\users\anton.shevchuk\appdata\local\programs\python\python39-32\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
